In [7]:
from sys import path
path.append('./src')
def run_tests(test_list):
    for test in test_list:
        try:
            test()
        except Exception as e:
            print(f"Exception: {test.__name__}: {e}")
            continue

In [8]:
from test.test_component_actions_dispatchers import *
tests = [test_dispatcher_ev_quit,
         test_dispatcher_initialization]

run_tests(tests)
# --- IGNORE ---

Exception: test_dispatcher_ev_quit: empty range in randrange(0, 0)


In [ ]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



In [ ]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [ ]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()
